In [33]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [34]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vila Franca Do Campo,37.7167,-25.4333,53.60,62,20,14.97,PT,2021-01-08 00:42:17
1,1,Nikolskoye,59.7035,30.7861,24.80,86,75,11.18,RU,2021-01-08 00:42:18
2,2,Rawson,-43.3002,-65.1023,60.39,53,0,10.29,AR,2021-01-08 00:40:47
3,3,Vostok,46.4856,135.8833,-18.76,82,100,3.71,RU,2021-01-08 00:42:18
4,4,Mataura,-46.1927,168.8643,62.01,72,66,4.00,NZ,2021-01-08 00:42:18


In [35]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [36]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [37]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [40]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,Hobart,-42.8794,147.3294,75.99,59,20,6.91,AU,2021-01-08 00:40:08
9,9,Busselton,-33.6500,115.3333,86.00,31,0,4.00,AU,2021-01-08 00:42:19
14,14,Banda Aceh,5.5577,95.3222,78.82,81,63,13.15,ID,2021-01-08 00:37:55
15,15,Kirakira,-10.4544,161.9205,84.29,75,100,8.30,SB,2021-01-08 00:42:20
16,16,Atuona,-9.8000,-139.0333,77.61,72,52,13.98,PF,2021-01-08 00:42:21
18,18,Carnarvon,-24.8667,113.6333,86.00,62,25,5.75,AU,2021-01-08 00:42:21
22,22,Rikitea,-23.1203,-134.9692,78.66,68,88,3.83,PF,2021-01-08 00:42:22
24,24,New Norfolk,-42.7826,147.0587,75.99,38,1,1.01,AU,2021-01-08 00:42:22
28,28,Arraial Do Cabo,-22.9661,-42.0278,77.00,91,0,5.75,BR,2021-01-08 00:37:27
29,29,Kapaa,22.0752,-159.3190,78.80,65,1,6.91,US,2021-01-08 00:42:23


In [41]:
preferred_cities_df.count()

City_ID       133
City          133
Lat           133
Lng           133
Max Temp      133
Humidity      133
Cloudiness    133
Wind Speed    133
Country       133
Date          133
dtype: int64

In [42]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Hobart,AU,75.99,-42.8794,147.3294,
9,Busselton,AU,86.00,-33.6500,115.3333,
14,Banda Aceh,ID,78.82,5.5577,95.3222,
15,Kirakira,SB,84.29,-10.4544,161.9205,
16,Atuona,PF,77.61,-9.8000,-139.0333,
18,Carnarvon,AU,86.00,-24.8667,113.6333,
22,Rikitea,PF,78.66,-23.1203,-134.9692,
24,New Norfolk,AU,75.99,-42.7826,147.0587,
28,Arraial Do Cabo,BR,77.00,-22.9661,-42.0278,
29,Kapaa,US,78.80,22.0752,-159.3190,


In [43]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key":g_key
}

In [44]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]


IndexError: list index out of range

In [ ]:
# Grab the first hotel from the results and store the name.
  try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
  except (IndexError):
    print("Hotel not found... skipping.")

In [ ]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""